In [1]:
%matplotlib inline

In [2]:
from __future__ import division
from scipy.io import wavfile
import stft_try as stft_try
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
s_sample, s = wavfile.read('data/trs.wav')
n_sample, n = wavfile.read('data/trn.wav')

x = s + n
print(x.shape)

(456707L,)


In [4]:
frame_size = 1024
hop_size = 512
window = np.hanning(frame_size)

s_spec = stft2(s, frame_size, hop_size, frame_size, window)[:513]
n_spec = stft2(n, frame_size, hop_size, frame_size, window)[:513]
x_spec = stft2(x, frame_size, hop_size, frame_size, window)[:513]
# s_spec = stft2(s)[:513]
# n_spec = stft2(n)[:513]
# x_spec = stft2(x)[:513]

In [5]:
def plsi(X, n=5000):
    B = np.random.random((X.shape[0], 30))
    theta = np.random.random((30, X.shape[1]))
    for _ in xrange(n):
        b_theta = np.dot(B, theta)
        B = np.multiply(B, np.dot(np.divide(X, b_theta), theta.T))
        one_v_v = np.ones((B.shape[0], B.shape[0]))
        B = np.divide(B, (np.dot(one_v_v, B)))
        b_theta = np.dot(B, theta)
        theta = np.multiply(theta, np.dot(B.T, np.divide(X, b_theta)))
        one_k_k = np.ones((theta.shape[0], theta.shape[0]))
        theta = np.divide(theta, (np.dot(one_k_k, theta)))
    return B, theta

In [6]:
B_s, theta_s = plsi(s_spec.real)

In [7]:
B_n, theta_n = plsi(n_spec.real)

In [8]:
B = np.hstack((B_s, B_n))
theta = np.vstack((theta_s, theta_n))
B.shape, theta.shape

((513L, 60L), (60L, 893L))

In [9]:
X = x_spec
Y = np.abs(X)
# Y = Y.T
Y.shape

(513L, 893L)

In [10]:
def update_theta(X, B, n=2000):
    theta = np.random.random((60, X.shape[1]))
    for _ in xrange(n):
        b_theta = np.dot(B, theta)
        theta = np.multiply(theta, np.dot(B.T, np.divide(X, b_theta)))
        one_k_k = np.ones((theta.shape[0], theta.shape[0]))
        theta = np.divide(theta, (np.dot(one_k_k, theta)))
    return theta

In [11]:
tex_sample, tex = wavfile.read('data/tex.wav')
tes_sample, tes = wavfile.read('data/tes.wav')

In [12]:
tex_spec = stft2(tex, frame_size, hop_size, frame_size, window)[:513]
# tes_spec = stft2(tes, frame_size, hop_size, frame_size, window)[:513]
tes_spec = stft2(tes)[:513]

In [13]:
theta_tex = update_theta(tex_spec.real, B)
theta_tex

array([[ -1.04608567e+001,  -5.55381729e-032,   6.45988865e+000, ...,
          2.96029480e+001,   0.00000000e+000,  -1.72246483e-003],
       [  8.62529082e+000,   2.88869040e+002,   1.72262378e-119, ...,
          4.99448549e+001,   3.99939306e+001,  -2.28457708e+001],
       [ -1.37864693e+001,  -1.68300983e+001,   8.53084144e+000, ...,
         -1.86798532e+002,   3.08335218e+001,   2.07103558e+002],
       ..., 
       [ -0.00000000e+000,  -0.00000000e+000,  -0.00000000e+000, ...,
         -0.00000000e+000,   0.00000000e+000,   0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000, ...,
          0.00000000e+000,  -0.00000000e+000,  -0.00000000e+000],
       [  0.00000000e+000,  -1.26171788e-004,  -2.80186849e+000, ...,
         -0.00000000e+000,   0.00000000e+000,   0.00000000e+000]])

In [14]:
numerator = np.dot(B_s, theta_tex[:30])

In [15]:
denominator = np.dot(B, theta_tex)

In [16]:
M_bar = numerator/denominator
M_bar.shape

(513L, 161L)

In [17]:
tex_sig = np.multiply(tex_spec, M_bar)

In [18]:
Y = tex_spec.real
Y = np.abs(Y)

In [19]:
tex_missing = tex_spec/Y

In [20]:
tex_sig += tex_missing

In [21]:
tex_recov = istft2(tex_sig)
# tex_recov = istft2(tes_spec)

In [22]:
wavfile.write('q3_out_wav_sig.wav', tes_sample, tex_recov)

In [23]:
def get_snr(X, X_recovered):
    numerator = np.sum(np.square(X))
    denominator = np.sum(np.square(X - X_recovered))
    print "numerator : ", numerator
    print "denominator : ", denominator
    return 10 * math.log10(numerator / denominator)

In [29]:
get_snr(tes_spec.real, tex_sig.real)

numerator :  5.968e+12
denominator :  3.60462922648e+14


-17.810317333299498